# Montando ambiente

In [17]:
import mysql.connector
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords


stop_words = set(stopwords.words('portuguese'))

# Configuração da conexão com o banco
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "graph_db"
}

"""Cria arestas com base em frases de exemplo."""
frases = [
        "Comece a gravação de áudio",
        "Salve esse texto importante",
        "Defina um alerta para amanhã",
        "Quero a gravação dessa reunião",
        "Salve a anotação rapidamente"
]

comandos = [
        "Iniciar gravação",
        "Salvar texto",
        "Configurar alerta",
        "Iniciar gravação",
        "Salvar anotação"
]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marcelo.maurin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Declara a funçao de conexao

In [9]:
def connect_to_database():
    """Retorna uma conexão com o banco de dados."""
    return mysql.connector.connect(**DB_CONFIG)

## Função Pega texto

In [3]:
def get_id(name, table):
    """Busca o ID de um item pelo nome, na tabela especificada."""
    connection = connect_to_database()
    cursor = connection.cursor()
    
    column = "word_name" if table == "words" else "command_name"
    query = f"SELECT * FROM {table} WHERE {column}=%s"
    cursor.execute(query, (name,))
    result = cursor.fetchone()
    
    cursor.close()
    connection.close()
    
    return result[0] if result else None

## Popula Arestas

In [4]:
def populaArestas(word1, word2, command):
    """Popula a tabela edges com as relações entre palavras e comando."""
    connection = connect_to_database()
    cursor = connection.cursor()
    
    word1_id = get_id(word1, 'words')
    word2_id = get_id(word2, 'words')
    command_id = get_id(command, 'commands')
    
    if not word1_id or not word2_id or not command_id:
        print(f"Erro: Palavras ou comando não encontrados: '{word1}', '{word2}', '{command}'")
        return
    
    insert_query = "INSERT INTO edges (word_id, command_id) VALUES (%s, %s)"
    
    # Inserindo relação da primeira palavra com o comando
    cursor.execute(insert_query, (word1_id, command_id))
    
    # Inserindo relação da segunda palavra com o comando
    cursor.execute(insert_query, (word2_id, command_id))
    
    connection.commit()
    cursor.close()
    connection.close()

## Cadastra Comandos

In [21]:
def cadastreCMD(command_name):
    """Cadastra um novo comando na tabela commands."""
    connection = connect_to_database()
    cursor = connection.cursor()
    
    insert_query = "INSERT INTO commands (command_name) VALUES (%s)"
    
    try:
        cursor.execute(insert_query, (command_name,))
        connection.commit()
    except mysql.connector.Error as err:
        print(f"Erro ao inserir comando '{command_name}': {err}")
    
    cursor.close()
    connection.close()

def ExemploCMD():   
    for cmd in comandos:
        cadastreCMD(cmd)

## Cadastra Palavras

In [37]:
def CadastraWords(word_name):
    """Cadastra uma nova palavra na tabela words."""
    connection = connect_to_database()
    cursor = connection.cursor()
    
    insert_query = "INSERT INTO words (word_name) VALUES (%s)"
    
    try:
        cursor.execute(insert_query, (word_name,))
        connection.commit()
    except mysql.connector.Error as err:
        print(f"Erro ao inserir palavra '{word_name}': {err}")
    
    cursor.close()
    connection.close()


def ExemploWords():
  for frase in frases:
        palavras = frase.split() # Separa a frase em palavras individuais
        for palavra in palavras:
            # Ignoramos palavras muito comuns (stop words) e números
            if palavra.lower() not in stop_words and not palavra.isnumeric():
                CadastraWords(palavra)


## Cria Arestas

In [34]:
def get_id_words(value, column_name):
    """Retorna o ID associado a um valor em uma coluna e tabela específicos."""
    connection = connect_to_database()
    cursor = connection.cursor()

    select_query = f"SELECT word_id FROM words WHERE {column_name} = %s"
    cursor.execute(select_query, (value,))

    result = cursor.fetchone()
    cursor.close()
    connection.close()

    return result[0] if result else None

def get_id_commands(value, column_name):
    """Retorna o ID associado a um valor em uma coluna e tabela específicos."""
    connection = connect_to_database()
    cursor = connection.cursor()

    select_query = f"SELECT command_id FROM commands WHERE {column_name} = %s"
    cursor.execute(select_query, (value,))

    result = cursor.fetchone()
    cursor.close()
    connection.close()

    return result[0] if result else None


def CriaArestas(Word1, Word2, Cmd):
    print(Word1)
    print(Word2)
    """Cria uma aresta associando Word1 e Word2 ao comando Cmd."""
    word1_id = get_id_words(Word1, "word_name")
    word2_id = get_id_words(Word2, "word_name")
    cmd_id = get_id_commands(Cmd,  "command_name")

    print(word1_id);
    print(word2_id);
    print(cmd_id);

    if not word1_id or not word2_id or not cmd_id:
        print("Erro: Um ou mais valores não foram encontrados no banco de dados.")
        return

    connection = connect_to_database()
    cursor = connection.cursor()
    
    insert_query = """
    INSERT INTO edges (word_id1, word_id2, command_id)
    VALUES (%s, %s, %s)
    """
    
    try:
        cursor.execute(insert_query, (word1_id, word2_id, cmd_id))
        connection.commit()
    except mysql.connector.Error as err:
        print(f"Erro ao inserir aresta: {err}")
    
    cursor.close()
    connection.close()

## Exemplo Arestas

In [35]:
def ExemploAresta():
    for index, frase in enumerate(frases):
        palavras = [palavra for palavra in frase.split() if palavra.lower() not in stop_words and not palavra.isnumeric()]
        
        for i in range(len(palavras) - 1):
            # Criamos arestas entre palavras sequenciais e associamos a um comando
            CriaArestas(palavras[i], palavras[i+1], comandos[index])


# Juntando Tudo

In [40]:
connect_to_database()

ExemploWords();

ExemploCMD();

ExemploAresta();

Comece
gravação
44
45
5
gravação
áudio
45
46
5
Salve
texto
47
48
6
texto
importante
48
49
6
Defina
alerta
50
51
7
alerta
amanhã
51
52
7
Quero
gravação
53
45
5
gravação
dessa
45
54
5
dessa
reunião
54
55
5
Salve
anotação
47
56
8
anotação
rapidamente
56
57
8
